In [1]:
import numpy as np
import pandas as pd
from glob import glob
import time
import itertools

from data_read import check_time_range
from talib import RSI, ATR

In [2]:
0.99**100

0.3660323412732292

In [2]:
def norm_rsi(rsi_array):
    rsi_array[rsi_array < 20] = 20
    rsi_array[rsi_array >= 81] = 80
    return rsi_array

In [3]:
data_path = 'G:/Code/Projects/Quant Mod/Robots/Data/BNB'
market = 'BNB-BTC'
kline  = '1h'
data_path = data_path +'/' +market + '/' + kline
fn = glob(data_path+'/*csv')[0]
df = pd.read_csv(fn);

#oneMin, fiveMin, thirtyMin, hour
buy_correction = 1.004
sell_correction = 0.996
deposit_percent = 0

print("Rows: ", len(df))
#print("Start time:", df.loc[0]['T'][:10])
#tr = check_time_range(df)
#tr = round(tr/3600/24)
#print("Дней:", tr)


#Initialize table with all zeros
n_states =  60 * 11
n_actions = 21
# Actions 30-nothing, 10-19 buy, 20-29 -sell 
Q = np.zeros([n_states, n_actions])
Q = pd.DataFrame(Q)
rsi_range = range(20,80)
states = itertools.product(rsi_range, range(11))
index = [''.join(map(str, x)) for x in states]
Q.index = index
buy_columns = list(range(1,11))
sell_columns = list(range(11,21))

#market position = 0, no tokens
for i in range(10):
    mp = [str(x) + str(i) for x in rsi_range]
    Q.loc[mp, sell_columns[i:]] = -np.inf #can't sell at this state   
#market position = 10, no capital
for i in range(10, 0,-1):
    mp = [str(x) +str(i) for x in rsi_range]
    Q.loc[mp, buy_columns[-i:]]  = -np.inf #can't buy at this state

#create lists to contain total rewards and steps per episode
rList = []

close_price = df['C'].as_matrix()
daily_returns = df['C'].pct_change()

rsi_window = 12
rsi = RSI(close_price * 100, rsi_window)
rsi[:rsi_window] = 0
rsi = np.round(rsi).astype(int)
rsi = norm_rsi(rsi)


Rows:  2145


In [4]:
class market_env_02:
    market_position = 0
    cur_capital = 0
    shares_amount = 0
        
    def step(self, j, a):
        contract = 0
        if a in buy_columns and close_price[j-1] != 0:
            contract = self.cur_capital / (11 - a)
            self.shares_amount = self.shares_amount + contract/close_price[j-1] * a
            self.cur_capital = self.cur_capital - contract * a
            self.market_position = self.market_position + a
        elif a in sell_columns and self.market_position != 0:
            contracts_to_sell = a - 10
            contract = self.shares_amount/self.market_position       
            self.shares_amount = self.shares_amount - contract * contracts_to_sell
            self.cur_capital = self.cur_capital + contract * contracts_to_sell * close_price[j-1]
            self.market_position = self.market_position - (a - 10)
            
        s = self.cur_state(j)
        state_capital = self.cur_capital + close_price[j] * self.shares_amount
        #print(a, contract, self.cur_capital, self.shares_amount)
        r = state_capital - start_capital
        return(s,r)
    
    def cur_state(self, t):
        state = str(rsi[t]) + str(self.market_position)      
        return state
   
    def restart(self):
        self.market_position = 0
        self.cur_capital = start_capital
        self.shares_amount = 0

In [5]:
class market_env_01:
    market_position = 0
        
    def step(self, j, a):

        if a in buy_columns:
             self.market_position = self.market_position + a
        elif a in sell_columns:
             self.market_position = self.market_position - (a - 10)

        r = daily_returns[j]
        if self.market_position == 0:
            r = -r
        else:
            r = r * self.market_position
        s = self.cur_state(j)
        return(s,r)
    
    def cur_state(self, t):
        state = str(rsi[t]) + str(self.market_position)      
        return state
   
    def restart(self):
        self.market_position = 0

In [5]:
# Set learning parameters
start_capital = 100
env = market_env_02()
env.cur_capital = start_capital
lr = .6
y = .94
num_episodes =  30
train = 0.7

start = rsi_window
train_end = df.shape[0] * train

start_time = time.time()
for i in range(num_episodes):
 
    rAll = 0
    j = start
    env.restart()
    #The Q-Table learning algorithm
    while j < train_end:
        #Choose an action by greedily (with noise) picking from Q table
        s = env.cur_state(j)
        a = np.argmax(Q.loc[s].values + np.random.randn(1, n_actions)*(1./(i+1)))
        #Get new state and reward from environment
        j = j + 1
        s1,r = env.step(j,  a)
        if np.isnan(r):
            break
        #Update Q-Table with new knowledge
        Q.loc[s,a] = Q.loc[s,a] + lr*(r + y*np.max(Q.loc[s1,:]) - Q.loc[s,a])
        rAll += r
        s = s1
    print(i, rAll)
    #rList.append(rAll)

print("Time taken = {0:.1f} sec".format(time.time() - start_time))    

0 -84588.63253737806
1 -57725.54646909596
2 32554.19070861278
3 59684.192201429876
4 59891.085128226194
5 59771.98095762074
6 43062.4000352739
7 60539.31681183107
8 40142.0585273844
9 40142.0585273844
10 40142.0585273844
11 40142.0585273844
12 40142.0585273844
13 40142.0585273844
14 40142.0585273844
15 40142.0585273844
16 40142.0585273844
17 40142.0585273844
18 40142.0585273844
19 40142.0585273844
20 40142.0585273844
21 40142.0585273844
22 40142.0585273844
23 40142.0585273844
24 40142.0585273844
25 40142.0585273844
26 40142.0585273844
27 40142.0585273844
28 40142.0585273844
29 40142.0585273844
Time taken = 39.6 sec


In [7]:
 Q.loc[:,11:];

In [8]:
market_position = 0
commision = 0.005

def cur_state(t): 
    state = str(rsi[t]) + str(market_position)      
    return state

start_capital = 100000
shares_amount = 0
num_trades = 0
contract = 0

cur_capital = start_capital
test_start = int(train_end) + 1
test_end = df.shape[0]
for i in range(test_start, test_end):
    s = cur_state(i)
    a = Q.loc[s,:].idxmax(axis = 1)
    if a in buy_columns:
        contracts_to_buy = a
        price = close_price[i] * buy_correction
        contract = cur_capital / (11 - market_position)     
        shares_amount = shares_amount + contract/price * contracts_to_buy * (1-commision)   
        cur_capital = cur_capital - contract * contracts_to_buy

        market_position = market_position + contracts_to_buy
        entryPrice = price            
        trade = 'entry'
    elif a in sell_columns:
        contracts_to_sell = a - 10
        price = close_price[i] * sell_correction
        contract = shares_amount/market_position
        profit = price * contract * contracts_to_sell * (1-commision)               
        
        shares_amount = shares_amount - contract * contracts_to_sell
        cur_capital = cur_capital + profit
        market_position = market_position - contracts_to_sell 
        trade = 'exit'
        num_trades += 1
    elif a == 0:
        price = close_price[i]
        trade='hold'
        
    cum_profit = shares_amount * close_price[i] + cur_capital
    print("%10s, %2s, %5.2f" % (trade, market_position, round(price,8)), round(shares_amount,2), round(cur_capital,3), cum_profit, s)

cum_profit = shares_amount * close_price[i] + cur_capital
print(cum_profit)

     entry,  1, 11184.70 0.81 90909.091 99918.50778703368 420
     entry,  5, 11194.61 4.04 54545.455 99600.52114585746 431
     entry, 10, 11209.21 8.08 9090.909 99251.81345150442 445
      exit,  3, 11100.91 2.42 71530.013 98532.1078298953 4310
     entry,  8, 11415.48 6.32 26823.755 98675.27320807835 603
     entry, 10, 11424.41 7.88 8941.252 98571.1513033694 608
      exit,  6, 11432.18 4.73 44781.008 99027.77827573016 6510
     entry, 10, 11494.80 7.83 8956.202 98569.15689582186 636
      exit,  0, 11373.60 0.0 97533.899 97533.89883143922 6010
     entry,  8, 11561.06 6.1 26600.154 96898.03857103868 650
      hold,  8, 11494.99 6.1 26600.154 96775.87959008088 638
      exit,  4, 11433.08 3.05 61324.558 96363.61158823397 628
      hold,  4, 11535.00 3.05 61324.558 96534.54869302249 654
      exit,  0, 11521.76 0.0 96318.28 96318.27995508551 674
     entry,  4, 11554.17 3.02 61293.451 96004.31236377955 610
      exit,  2, 11483.88 1.51 78525.701 95914.09870164943 624
     entry,  9,

In [ ]:
Q.iloc[510:,]

In [9]:
cur_capital

1.256513474840008e-65